<a href="https://colab.research.google.com/github/nailimmnh/PORTOFOLIO/blob/main/MachineLearningPortofolio/Sentiment%20Analysis%20FinancialBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Nailiyatul Mamnunah

NPM : 2106722801

Tugas Week 7 - Komputasi Intelegensia

# **FinancialBERT for Sentiment Analysis**

source : https://huggingface.co/ahmedrachid/FinancialBERT-Sentiment-Analysis#training-data

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ahmedrachid/FinancialBERT-Sentiment-Analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/464k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

## 1. Menguji Model dengan Input Sintetik atau Kalimat Asli

In [ ]:
# Contoh kalimat untuk dianalisis
sentences = [
    "The company's stock price increased by 10% after the positive earnings report.",
    "The financial performance of the company is very poor this quarter."
]

# Menggunakan pipeline untuk memprediksi sentimen
results = pipe(sentences)

# Menampilkan hasil prediksi
for sentence, result in zip(sentences, results):
    print(f"Text: {sentence}")
    print(f"Sentiment: {result['label']}, Confidence: {result['score']:.4f}")
    print()


Text: The company's stock price increased by 10% after the positive earnings report.
Sentiment: positive, Confidence: 0.9998

Text: The financial performance of the company is very poor this quarter.
Sentiment: negative, Confidence: 0.9993



**Penjelasan**

* Kalimat 1: "The company's stock price increased by 10% after the positive earnings report."
* Kalimat 2: "The financial performance of the company is very poor this quarter."

Pipeline memberikan prediksi berikut:

* Kalimat pertama: Sentiment positif dengan confidence (keyakinan model) 0.9998.
* Kalimat kedua: Sentiment negatif dengan confidence 0.9993.

Ini menunjukkan bahwa model BERT mengenali sentimen positif pada kalimat yang berbicara tentang peningkatan harga saham setelah laporan laba, dan sentimen negatif pada kalimat yang berbicara tentang kinerja keuangan yang buruk.

## 2. Menambahkan Synthetic Dataset (untuk Pelatihan Tambahan)

In [ ]:
import torch
synthetic_sentences = [
    "The profit margin has decreased significantly.",
    "This is the best quarter for the company in the last decade.",
    "The company will face major losses if the stock market continues to drop.",
    "Their financial strategy is proving to be very successful."
]

synthetic_labels = [0, 1, 0, 1]  # 0 = Negative, 1 = Positive

# Tokenize the dataset
inputs = tokenizer(synthetic_sentences, padding=True, truncation=True, return_tensors="pt")

# Adding the labels to inputs
inputs['labels'] = torch.tensor(synthetic_labels)

# Menggunakan optimizer untuk fine-tuning
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Fine-tuning loop
model.train()
for epoch in range(3):  # Sesuaikan jumlah epoch jika diperlukan
    outputs = model(**inputs)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch + 1} Loss: {loss.item()}")


Epoch 1 Loss: 4.412972927093506
Epoch 2 Loss: 2.6945228576660156
Epoch 3 Loss: 2.0907678604125977


**Penjelasan**

menggunakan dataset sintetik yang berisi empat kalibat dengan label sentiment:

* Kalimat-kalimat:

  * "The profit margin has decreased significantly." (Negatif)
  * "This is the best quarter for the company in the last decade." (Positif)
  * "The company will face major losses if the stock market continues to drop." (Negatif)
  * "Their financial strategy is proving to be very successful." (Positif)

Model dilatih menggunakan AdamW optimizer selama 3 epoch.
Hasil dari pelatihan (loss):

1. Epoch 1 Loss: 4.41
2. Epoch 2 Loss: 2.69
3. Epoch 3 Loss: 2.09

Penurunan nilai loss dari epoch ke epoch menunjukkan bahwa model berhasil belajar dari dataset sintetik dan menjadi lebih baik dalam mengklasifikasikan sentimen.

## 3. Evaluasi Model Setelah Fine-Tuning

In [ ]:
# Contoh tes setelah fine-tuning
test_sentences = [
    "The company is facing severe financial issues.",
    "They had a very profitable year."
]

test_inputs = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    test_outputs = model(**test_inputs)
    predictions = torch.argmax(test_outputs.logits, dim=-1)

# Menampilkan hasil prediksi
for sentence, pred in zip(test_sentences, predictions):
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Text: {sentence} -> Sentiment: {sentiment}")


Text: The company is facing severe financial issues. -> Sentiment: Negative
Text: They had a very profitable year. -> Sentiment: Positive


**Penjelasan**

Menguji model dengan dua kalimat baru:

* Kalimat 1: "The company is facing severe financial issues." (Diharapkan negatif)
* Kalimat 2: "They had a very profitable year." (Diharapkan positif)

Prediksi dari model setelah fine-tuning:

* Kalimat pertama diprediksi sebagai Sentiment Negatif.
* Kalimat kedua diprediksi sebagai Sentiment Positif.

Hasil ini sesuai dengan harapan, yang menunjukkan bahwa fine-tuning dengan dataset sintetik berhasil memperbaiki model dalam mengenali sentimen pada kalimat baru.

## 3. Attention Transformer

karena model BERT sendiri sudah mengimplementasikan **self-attention mechanism** di dalamnya, maka tidak perlu mengimplentasikan attention secara terpisah karena BERT telah di latih dengan mekanisme attention.